# Домашна задача 3 - Имплементација на ECB и CBC модови и „Исечи па залепи“ напад кај двата мода. 
Милена Кукољ 186085 

## Имплементација на ECB и CBC модовите

In [1]:
from des import DesKey

In [2]:
key0 = DesKey(b"some key")                  # for DES

Во оваа библиотека има една класа DesKey. Откако ќе се креира клучот, кој мора да биде во бајти и со должина од 8, 16 или 24 бајти. Од должината на клучот зависи и кој алгоритам ќе се користи понатаму. Доколку е 16 или 24 бајти, се користи 3DES алгоритамот. Кој алгоритам е искористен можеме да провериме со методот *is_single*.

In [3]:
key0.is_single() 

True

In [4]:
key0.encrypt(b"any long message") 

b"\x14\xfa\xc2 '\x00{\xa9\xdc;\x9dq\xcbr\x87Q"

Ако во методот за енкриптирање на порака не внесеме вредост за иницијалниот вектор, ќе се употреби модот CBC, како на примерот погоре. Во спротивно, ќе се употреби ECB модот на енкрипција. 

In [5]:
key0.encrypt(b"any long message", initial=0) 

b"\x14\xfa\xc2 '\x00{\xa9\xb2\xa5\xa7\xfb#\x86\xc5\x9b"

Оваа имплементација има и опција за додавање на дополнителни битови на крај од пораката доколку е со должина на бајти која не е делива со 8.

In [6]:
key0.encrypt(b"abc", padding=True) 

b'%\xd1KU\x8b_A\xa6'

In [7]:
key0.decrypt(b"%\xd1KU\x8b_A\xa6", padding=True) 

b'abc'

Откако ќе се повика методот *encrypt* тој ја повикува функцијата *handle* каде се одредува и кој метод ќе се користи за енкрипцијата, според вредноста на променливата *initial*. 

In [8]:
def handle(message, key, initial, padding, encryption):
    message = guard_message(message, padding, encryption)
    initial = guard_initial(initial)

    blocks = (struct.unpack(">Q", message[i: i + 8])[0] for i in iter_range(0, len(message), 8))

    if initial is None:
        # ECB
        encoded_blocks = ecb(blocks, key, encryption)
    else:
        # CBC
        encoded_blocks = cbc(blocks, key, initial, encryption)

    ret = b"".join(struct.pack(">Q", block) for block in encoded_blocks)
    return ret[:-ord(ret[-1:])] if not encryption and padding else ret

### ECB

In [9]:
def ecb(blocks, key, encryption):
    for block in blocks:
        yield encode(block, key, encryption)

In [13]:
def encode(block, key, encryption):
    for k in key:
        block = encode_block(block, k, encryption)
        encryption = not encryption # important for longer keys 3DES

    return block

In [11]:
def encode_block(block, derived_keys, encryption):
    block = permute(block, 64, INITIAL_PERMUTATION)
    block = block >> 32, block & 0xffffffff

    if not encryption:
        derived_keys = reversed(derived_keys)
    for key in derived_keys:
        block = block[1], block[0] ^ f(block[1], key)

    return permute(block[1] << 32 | block[0], 64, INVERSE_PERMUTATION)

### CBC

In [12]:
def cbc(blocks, key, initial, encryption):
    if encryption:
        for block in blocks:
            initial = encode(block ^ initial, key, encryption)
            yield initial
    else:
        for block in blocks:
            initial, block = block, initial ^ encode(block, key, encryption)
            yield block

Всушност главната разлика во модовите на екрипција е дека во CBC блокот вредноста што се праќа на методот *encode* е резултатот од XOR операцијата на моменталниот блок и променливата *initial*. Променливата *initial* првично ја има вредноста на иницијалниот вектор, но потоа и ја доделуваме вредноста од енкриптираниот блок.

При декрипција на првиот блок, се доделува резултатот од XOR операцијата на иницијалниот вектор и декриптираниот блок на променливата *block*, додека на *initial* променливата и се доделува енкриптираниот блок. На овој начин се продолжува за секој од блоковите.

## „Исечи па залепи“ напад кај двата мода. 

In [14]:
def print_results(plain_text, swap_text, delete_text):
    print("====================")
    print("Plain text:")
    print("====================")
    print(bank_transfer)
    print("====================")
    print("Change decrypted message:")
    print("====================")
    print(plain_text_ecb_swap.decode())
    print("====================")
    print("Delete decrypted message:")
    print("====================")
    print(plain_text_ecb_delete)

In [19]:
# the fields in the bank transfer are 8 bytes long
# 0. sending bank's ID,
# 1. account number at sending bank,
# 2. receiving bank's ID,
# 3. account number at receiving bank,
# 4. amount transferred
bank_transfer = b"BANK1234ACCT5678BANK5678ACCT123400010000"

### ECB мод

In [15]:
def substitution_ecb(message, change_blocks=[], delete_blocks=[]):
    message = bytearray(message)

    if len(delete_blocks) > 0:
        for delete in delete_blocks:
            message.pop(delete)

    for ((a, b), (c, d)) in change_blocks:
        message[a:b], message[c:d] = message[c:d], message[a:b]

    return bytes(message)

Оваа функција едноставно само им ги заменува местата на блоковите, или ги брише целосно од поракта.

In [20]:
encrypted_ecb = key0.encrypt(bank_transfer)

swap_message_ecb = substitution_ecb(encrypted_ecb, change_blocks=[((0,8), (8,16))])
plain_text_ecb_swap = key0.decrypt(swap_message_ecb)

delete_message_ecb = substitution_ecb(encrypted_ecb, delete_blocks=[0,1,2,4,5,6,7,8])
plain_text_ecb_delete = key0.decrypt(delete_message_ecb)

print_results(bank_transfer, plain_text_ecb_swap, plain_text_ecb_delete)

Plain text:
b'BANK1234ACCT5678BANK5678ACCT123400010000'
Change decrypted message:
ACCT5678BANK1234BANK5678ACCT123400010000
Delete decrypted message:
b'\xd4m\xea^\x8b\xf9\xcc\rBANK5678ACCT123400010000'


### CBC мод

In [56]:
def xor_bytes(a, b):
    return bytes(x ^ y for x, y in zip(a, b))

In [96]:
def substitution_cbc(original_message, modified_message=b"", change_block=1, delete_blocks=[]):
    original_message = bytearray(original_message)
    if len(delete_blocks) > 0:
        for delete in delete_blocks:
            original_message.pop(delete)
        return bytes(original_message)
    else:
        if change_block == 0:
            offset = change_block * 8
            before_block = original_message[offset - 8: offset]
            after_block = original_message[offset:]
            # find what to change
            xor_diff = xor_bytes(original_message, modified_message)
            before_block = b""
            fake_iv = xor_bytes(b"\0"*8, xor_diff)
            plain_text_cbc_change = key0.decrypt(bytes(original_message), initial=fake_iv)
        else:
            offset = change_block * 8
            before_block = original_message[offset - 8: offset]
            after_block = original_message[offset:]
            # find what to change
            xor_diff = xor_bytes(original_message, modified_message)
            # make the change in the previous block
            modified_before_block = xor_bytes(before_block, xor_diff)
            changed_message = bytes(original_message[:offset-8] + modified_before_block + after_block)
            print(changed_message)
            plain_text_cbc_change = key0.decrypt(changed_message, initial=0)
        return plain_text_cbc_change

Оваа функција или брише дел од пораката, или го менува претходниот дел од блокот за да влијае при декрипција на следниот дел од блокот. Функцијата прима како параметри оригиналната енкриптирана порака што сме ја добиле, модифицираната порака и во кој блок сакаме да ја направи промената, и опционалниот параметар ** ако сакаме да избришеме дел од пораката.

In [102]:
bank_transfer     = b"BANK1234ACCT1234BANK5678ACCT123400010000"
bank_transfer_mod = b"BANK1234ACCT1234BANK5678ACCT444400010000"
print(xor_bytes(bank_transfer, bank_transfer_mod))

encrypted_cbc = key0.encrypt(bank_transfer, initial=0)
changed_encr_message_cbc = key0.encrypt(bank_transfer_mod, initial=0)

plain_text_cbc_change = substitution_cbc(encrypted_cbc, changed_encr_message_cbc, change_block=3)

delete_message_cbc = substitution_cbc(encrypted_cbc, delete_blocks=[0,1,2,4,5,6,7,8])
plain_text_cbc_delete = key0.decrypt(delete_message_cbc, initial=0)

print_results(bank_transfer, plain_text_cbc_change, plain_text_cbc_delete)

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x06\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xe4\x0c\x89\xechH7\x061nA\xf3IB\xe3\xd7+\xcf\xe7C\xda\xc6\xb6O\x82\xee\x19\x94\xcb\xf6c\x8a|\x9e\xb6\xa5F\xb8C\x06'
Plain text:
b'BANK1234ACCT1234BANK5678ACCT123400010000'
Change decrypted message:
ACCT5678BANK1234BANK5678ACCT123400010000
Delete decrypted message:
b'\xd4m\xea^\x8b\xf9\xcc\rBANK5678ACCT123400010000'
